In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/MyDrive/xAI\ for\ Social\ Bias

/content/drive/MyDrive/xAI for Social Bias


In [3]:
import pandas as pd
import numpy as np

In [4]:
twitter=pd.read_csv('Twitter_Data.csv')

In [5]:
twitter.isnull().sum()

clean_text    4
category      7
dtype: int64

In [6]:
twitter.dropna(axis=0, inplace=True)

In [7]:
train=[]
train=list(twitter['clean_text'])

labels=[]
labels=list(twitter['category'])

In [8]:
df = twitter
df['category'] = df['category'].map({-1.0:'Negative', 0.0:'Neutral', 1.0:'Positive'})
# Output first five rows
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,Negative
1,talk all the nonsense and continue all the dra...,Neutral
2,what did just say vote for modi welcome bjp t...,Positive
3,asking his supporters prefix chowkidar their n...,Positive
4,answer who among these the most powerful world...,Positive


In [9]:
import re    # RegEx for removing non-letter characters

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *


def tweet_to_words(tweet):
    ''' Convert tweet text into a sequence of words '''
    
    # convert to lowercase
    text = tweet.lower()
    # remove non letters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    words = [PorterStemmer().stem(w) for w in words]
    # return list
    return words

print("\nOriginal tweet ->", df['clean_text'][0])
print("\nProcessed tweet ->", tweet_to_words(df['clean_text'][0]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!

Original tweet -> when modi promised “minimum government maximum governance” expected him begin the difficult job reforming the state why does take years get justice state should and not business and should exit psus and temples

Processed tweet -> ['modi', 'promis', 'minimum', 'govern', 'maximum', 'govern', 'expect', 'begin', 'difficult', 'job', 'reform', 'state', 'take', 'year', 'get', 'justic', 'state', 'busi', 'exit', 'psu', 'templ']


In [10]:
X = list(map(tweet_to_words, df['clean_text']))

In [11]:
train_full = [' '.join(words) for words in X]

In [12]:
from sklearn.preprocessing import LabelEncoder

# Encode target labels
le = LabelEncoder()
Y = le.fit_transform(df['category'])

In [13]:
print(X[0])
print(Y[0])

['modi', 'promis', 'minimum', 'govern', 'maximum', 'govern', 'expect', 'begin', 'difficult', 'job', 'reform', 'state', 'take', 'year', 'get', 'justic', 'state', 'busi', 'exit', 'psu', 'templ']
0


In [14]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(X)*split_frac)
train_x, test_x = X[:split_idx], X[split_idx:]
train_y, test_y = Y[:split_idx], Y[split_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(len(train_x)), 
      "\nTest set: \t\t{}".format(len(test_x)))

			Feature Shapes:
Train set: 		130375 
Test set: 		32594


# BAG of WORDS(Feature Extraction)

In [15]:
#Changing list into a string
lower = [' '.join(words) for words in train_x]
lower_test = [' '.join(words) for words in test_x]

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

# Tweets have already been preprocessed hence dummy function will be passed in 
# to preprocessor & tokenizer step
count_vector = CountVectorizer()

# Fit the training data
cv_train = count_vector.fit_transform(lower)

# Transform testing data
cv_test = count_vector.transform(lower_test)

In [17]:
print('Bag of Words_cv_train:',cv_train.shape)
print('Bag of Words_cv_test:',cv_test.shape)

Bag of Words_cv_train: (130375, 74590)
Bag of Words_cv_test: (32594, 74590)


# TF-IDF Feature

In [18]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Create a Vectorizer Object using default parameters
hash_vectorizer = HashingVectorizer(alternate_sign=False)

# Convert a collection of text documents to a matrix of token counts
token_count_matrix=hash_vectorizer.fit_transform(train_full)
print(f'The size of the count matrix for the texts = {token_count_matrix.get_shape()}')
print(f'The sparse count matrix is as follows:')
print(token_count_matrix)

# Create a tf_idf object using default parameters
tf_idf_transformer=TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=False) 

# Fit to the count matrix, then transform it to a normalized tf-idf representation
tf_idf_matrix = tf_idf_transformer.fit_transform(token_count_matrix)

print(f'The size of the tf_idf matrix for the texts = {tf_idf_matrix.get_shape()}')
print(f'The sparse tf_idf matrix is as follows:')
print(tf_idf_matrix)

The size of the count matrix for the texts = (162969, 1048576)
The sparse count matrix is as follows:
  (0, 106270)	0.2
  (0, 109026)	0.4
  (0, 192080)	0.2
  (0, 245582)	0.2
  (0, 263274)	0.2
  (0, 277794)	0.2
  (0, 288398)	0.2
  (0, 300083)	0.2
  (0, 360502)	0.4
  (0, 369935)	0.2
  (0, 387101)	0.2
  (0, 433698)	0.2
  (0, 449993)	0.2
  (0, 614924)	0.2
  (0, 661528)	0.2
  (0, 831430)	0.2
  (0, 865698)	0.2
  (0, 997510)	0.2
  (0, 1011271)	0.2
  (1, 15564)	0.4082482904638631
  (1, 72884)	0.4082482904638631
  (1, 433698)	0.4082482904638631
  (1, 455884)	0.4082482904638631
  (1, 540272)	0.4082482904638631
  (1, 600238)	0.4082482904638631
  :	:
  (162967, 597420)	0.30151134457776363
  (162967, 628922)	0.30151134457776363
  (162967, 743937)	0.30151134457776363
  (162967, 788840)	0.30151134457776363
  (162967, 857707)	0.30151134457776363
  (162967, 929792)	0.6030226891555273
  (162968, 60869)	0.1889822365046136
  (162968, 77761)	0.1889822365046136
  (162968, 151348)	0.1889822365046136
  (16296

In [19]:
#Splitting the data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tf_idf_matrix, labels,
                                                    test_size=0.2)

# MODELS

In [20]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

## Naive Bayes

### BOW

In [23]:
mnb=MultinomialNB()
#fitting the svm for bag of words
mnb_bow=mnb.fit(cv_train,train_y)
print(mnb_bow)

#Predicting the model for bow features
mnb_bow_predict=mnb.predict(cv_test)
print(mnb_bow_predict)

mnb_bow_score=accuracy_score(test_y,mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
[0 1 2 ... 2 2 2]
mnb_bow_score : 0.68687488494815


In [34]:
import pickle
Filename = "NB.pkl"  
with open(Filename, 'wb') as file:  
    pickle.dump(mnb, file)

### TF-IDF

In [24]:
mnb=MultinomialNB()
#fitting the svm for tfidf features
mnb_tfidf=mnb.fit(X_train,y_train)
print(mnb_tfidf)

# Predicting the model for tfidf features
mnb_tfidf_predict=mnb.predict(X_test)
print(mnb_tfidf_predict)

mnb_tfidf_score=accuracy_score(y_test,mnb_tfidf_predict)
print(f"mnb tfidf accuracy: {mnb_tfidf_score * 100:.3f}%", )

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
[1. 1. 1. ... 1. 1. 1.]
mnb tfidf accuracy: 47.045%


## Logistic Regression

### BOW

In [35]:
#training the model
lr=LogisticRegression(penalty='l2',solver='lbfgs', max_iter=600,C=1,random_state=42)
#Fitting the model for Bag of words
lr_bow=lr.fit(cv_train,train_y)
print(lr_bow)

lr_bow_predict=lr.predict(cv_test)
print(lr_bow_predict)

lr_bow_score=accuracy_score(test_y,lr_bow_predict)
print("lr_bow_score :",lr_bow_score)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=600,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
[0 1 2 ... 1 1 2]
lr_bow_score : 0.8356752776584647


In [36]:
import pickle
Filename = "Logistic_R.pkl"  
with open(Filename, 'wb') as file:  
    pickle.dump(lr, file)

### TF-IDF

In [26]:
#training the model
lr=LogisticRegression(penalty='l2',solver='lbfgs', max_iter=600,C=1,random_state=42)

# Fitting the model for tfidf features
lr_tfidf=lr.fit(X_train,y_train)
print(lr_tfidf)

lr_tfidf_predict=lr.predict(X_test)
print(lr_tfidf_predict)

lr_tfidf_score=accuracy_score(y_test,lr_tfidf_predict)
print(f"Logistic Regression tfidf Accuracy: {lr_tfidf_score * 100:.3f}%", )

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=600,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
[ 1.  0. -1. ...  1.  1. -1.]
Logistic Regression tfidf Accuracy: 84.019%


## SVM Classifiers(Both SGD Trained) 

### BOW

In [37]:
#training the linear svm
svm=SGDClassifier(loss='hinge',max_iter=500,random_state=42)

#fitting the svm for bag of words
svm_bow=svm.fit(cv_train,train_y)
print(svm_bow)

svm_bow_predict=svm.predict(cv_test)
print(svm_bow_predict)

#Accuracy score for bag of words
svm_bow_score=accuracy_score(test_y,svm_bow_predict)
print("svm_bow_score :",svm_bow_score)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=500, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)
[0 1 2 ... 1 1 2]
svm_bow_score : 0.8430385960606247


### TF-IDF

In [28]:
#training the linear svm
svm=SGDClassifier(loss='hinge',max_iter=500,random_state=42)

# #fitting the svm for tfidf features
svm_tfidf=svm.fit(X_train,y_train)
print(svm_tfidf)

#Predicting the model for tfidf features
svm_tfidf_predict=svm.predict(X_test)
print(svm_tfidf_predict)

#Accuracy score for tfidf features
svm_tfidf_score=accuracy_score(y_test,svm_tfidf_predict)
print(f"SVM tfidf accuracy: {svm_tfidf_score * 100:.3f}%", )

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=500, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)
[1. 0. 0. ... 0. 1. 1.]
SVM tfidf accuracy: 80.128%


In [38]:
import pickle
Filename = "SVM_SGD.pkl"  
with open(Filename, 'wb') as file:  
    pickle.dump(svm, file)

## SVM Classifier( Without SGD )

### BOW

In [39]:
from sklearn.svm import LinearSVC
#Creating, fitting and scoring classifier
classifier = LinearSVC(max_iter=10000, dual=True)
classifier.fit(cv_train, train_y)
print(f"BOW Accuracy: {classifier.score(cv_test, test_y) * 100:.3f}%", )

BOW Accuracy: 82.469%


In [40]:
import pickle
Filename = "Linear_SVM_BOW.pkl"  
with open(Filename, 'wb') as file:  
    pickle.dump(classifier, file)

### TF-IDF

In [41]:
from sklearn.svm import LinearSVC
#Creating, fitting and scoring classifier
classifier = LinearSVC(max_iter=1000, dual=True)
classifier.fit(X_train,y_train)
print(f"TF-IDF Accuracy: {classifier.score(X_test, y_test) * 100:.3f}%", )

TF-IDF Accuracy: 84.571%


In [42]:
import pickle
Filename = "Linear_SVM_TFIDF.pkl"  
with open(Filename, 'wb') as file:  
    pickle.dump(classifier, file)

In [31]:
# # Model 2 - RandomForest Classifier
# from sklearn.ensemble import RandomForestClassifier

# model_rf = RandomForestClassifier()
# model_rf.fit(cv_train1, train_y)
# rf_score=accuracy_score(test_y, model_rf.predict(cv_test))
# print("rf_bow_score :",rf_score)

In [32]:
# # Model 3 - XGB Classifier
# from xgboost import XGBClassifier

# model_xgb = XGBClassifier()
# model_xgb.fit(cv_train, train_y)
# xgb_score=accuracy_score(test_y, model_xgb.predict((cv_test))
# print("xgb_bow_score :",xgb_score)